In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /root/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /root/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /root/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /root/nltk_data...
 

True

In [0]:

!python /content/drive/My\ Drive/Attention-Based-Aspect-Extraction-master/code/preprocess.py 

!python /content/drive/My\ Drive/Attention-Based-Aspect-Extraction-master/code/word2vec.py 


	restaurant train set ...
	restaurant test set ...
Pre-training word embeddings ...


In [0]:
f = open("/content/drive/My Drive/Attention-Based-Aspect-Extraction-master/datasets/restaurant/train.txt",encoding = 'utf-8')
len(f.read())

235446

In [0]:
!pip install pymorphy2
#!pip uninstall tensorflow
#!pip install tensorflow-gpu
#!pip uninstall keras
#!pip uninstall tensorflow-gpu
#!pip install tensorflow-gpu
#!pip install keras==2.3

import sys
import re
import pymorphy2
import os
import numpy as np
import logging
import keras.backend as K
import json
import gensim
import errno
import codecs
import argparse

import pandas as pd
from tqdm import tqdm
from time import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from keras.preprocessing import sequence
from keras.models import Model
from keras.layers import Dense, Activation, Embedding, Input
from keras.engine.topology import Layer
from keras.constraints import MaxNorm
from keras import regularizers
from keras import initializers
from keras import constraints



import reader as dataset 
import utils2 as U
from model import create_model
from w2vEmbReader import W2VEmbReader as EmbReader
from my_layers import Attention, Average, WeightedSum, WeightedAspectEmb, MaxMargin

import keras.optimizers as opt

def get_optimizer(args):
	clipvalue = 0
	clipnorm = 10

	if args.algorithm == 'rmsprop':
		optimizer = opt.RMSprop(lr=0.001, rho=0.9, epsilon=1e-06, clipnorm=clipnorm, clipvalue=clipvalue)
	elif args.algorithm == 'sgd':
		optimizer = opt.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False, clipnorm=clipnorm, clipvalue=clipvalue)
	elif args.algorithm == 'adagrad':
		optimizer = opt.Adagrad(lr=0.01, epsilon=1e-06, clipnorm=clipnorm, clipvalue=clipvalue)
	elif args.algorithm == 'adadelta':
		optimizer = opt.Adadelta(lr=1.0, rho=0.95, epsilon=1e-06, clipnorm=clipnorm, clipvalue=clipvalue)
	elif args.algorithm == 'adam':
		optimizer = opt.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, clipnorm=clipnorm, clipvalue=clipvalue)
	elif args.algorithm == 'adamax':
		optimizer = opt.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, clipnorm=clipnorm, clipvalue=clipvalue)
	
	return optimizer


In [0]:
import tensorflow as tf
tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)

True

In [0]:
#### training 
    
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
logger = logging.getLogger(__name__)

test_len = 50000
version = 8 
trainable = 1  


### set parameters
parser = U.add_common_args()
parser.add_argument("-e", "--embdim", dest="emb_dim", type=int, metavar='<int>', default=200,
                    help="Embeddings dimension (default=200)")
parser.add_argument("-as", "--aspect-size", dest="aspect_size", type=int, metavar='<int>', default=30,
                    help="The number of aspects specified by users (default=14)") 
parser.add_argument("--emb-name",  type=str,
                    help="The name to the word embeddings file", default="sample_w2v_embedding")
parser.add_argument("--epochs", dest="epochs", type=int, metavar='<int>', default=15,
                    help="Number of epochs (default=15)")
parser.add_argument("-n", "--neg-size", dest="neg_size", type=int, metavar='<int>', default=20,
                    help="Number of negative instances (default=20)")
parser.add_argument("--seed", dest="seed", type=int, metavar='<int>', default=1234,
                    help="Random seed (default=1234)")
parser.add_argument("-a", "--algorithm", dest="algorithm", type=str, metavar='<str>', default='adam',
                    help="Optimization algorithm (rmsprop|sgd|adagrad|adadelta|adam|adamax) (default=adam)")
parser.add_argument("--ortho-reg", dest="ortho_reg", type=float, metavar='<float>', default=0.1,
                    help="The weight of orthogonal regularization (default=0.1)")

args = parser.parse_args(args=["--aspect-size",'6','--emb-name','w2v_embedding',"--domain",'restaurant',"--epochs",'15'])
# args = parser.parse_args(args=[])  
print(args)

out_dir = args.out_dir_path + '/' + args.domain
U.mkdir_p(out_dir)
U.print_args(args)

assert args.algorithm in {'rmsprop', 'sgd', 'adagrad', 'adadelta', 'adam', 'adamax'}
# assert args.domain in {'restaurant', 'beer'}

if args.seed > 0:
  np.random.seed(args.seed)

2019-12-05 21:17:13,039 INFO Arguments:
2019-12-05 21:17:13,040 INFO   algorithm: adam
2019-12-05 21:17:13,041 INFO   aspect_size: 6
2019-12-05 21:17:13,042 INFO   batch_size: 32
2019-12-05 21:17:13,044 INFO   command: /usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-bfaf3b35-21f0-4e60-911c-865a2c381336.json
2019-12-05 21:17:13,045 INFO   domain: restaurant
2019-12-05 21:17:13,046 INFO   emb_dim: 200
2019-12-05 21:17:13,047 INFO   emb_name: w2v_embedding
2019-12-05 21:17:13,048 INFO   epochs: 15
2019-12-05 21:17:13,050 INFO   maxlen: 256
2019-12-05 21:17:13,051 INFO   neg_size: 20
2019-12-05 21:17:13,053 INFO   ortho_reg: 0.1
2019-12-05 21:17:13,054 INFO   out_dir_path: output
2019-12-05 21:17:13,055 INFO   seed: 1234
2019-12-05 21:17:13,057 INFO   vocab_size: 9000


Namespace(algorithm='adam', aspect_size=6, batch_size=32, domain='restaurant', emb_dim=200, emb_name='w2v_embedding', epochs=15, maxlen=256, neg_size=20, ortho_reg=0.1, out_dir_path='output', seed=1234, vocab_size=9000)


In [0]:
vocab, train_x, test_x, overall_maxlen = dataset.get_data(args.domain, vocab_size=args.vocab_size, maxlen=args.maxlen) 
train_x = sequence.pad_sequences(train_x, maxlen=overall_maxlen)
test_x = sequence.pad_sequences(test_x, maxlen=overall_maxlen)

print('Total Number of training examples: ', len(train_x))

train_x = train_x[0:test_len] 
print('Number of training examples: ', len(train_x))
print('Length of vocab: ', len(vocab))

Reading data from restaurant
 Creating vocab ...
   23707 total words, 2956 unique words
  keep the top 9000 words
 Reading dataset ...
  train set
   <num> hit rate: 0.00%, <unk> hit rate: 0.00%
  test set
   <num> hit rate: 0.41%, <unk> hit rate: 56.73%
Total Number of training examples:  2603
Number of training examples:  2603
Length of vocab:  2959


In [0]:
# train.py --- 

###############################################################################################################################
def sentence_batch_generator(data, batch_size):
    n_batch = len(data) // batch_size
    batch_count = 0
    np.random.shuffle(data)

    while True:
        if batch_count == n_batch:
            np.random.shuffle(data)
            batch_count = 0

        batch = data[batch_count * batch_size: (batch_count + 1) * batch_size]
        batch_count += 1
        yield batch


def negative_batch_generator(data, batch_size, neg_size):
    data_len = data.shape[0]
    dim = data.shape[1]

    while True:
        indices = np.random.choice(data_len, batch_size * neg_size)
        samples = data[indices].reshape(batch_size, neg_size, dim)
        yield samples


###############################################################################################################################
## Optimizaer algorithm
optimizer = get_optimizer(args) 

###############################################################################################################################
## Building model



logger.info('  Building model')
model = create_model(args, overall_maxlen, vocab)
# freeze the word embedding layer
model.get_layer('word_emb').trainable = bool(trainable)   
model.compile(optimizer=optimizer, loss=U.max_margin_loss, metrics=[U.max_margin_loss]) 

###############################################################################################################################
## Training


logger.info("-"*80)

vocab_inv = {}
for w, ind in vocab.items():
    vocab_inv[ind] = w

sen_gen = sentence_batch_generator(train_x, args.batch_size) 
neg_gen = negative_batch_generator(train_x, args.batch_size, args.neg_size)
batches_per_epoch = len(train_x) // args.batch_size

min_loss = float('inf')
for ii in range(args.epochs):
    t0 = time()
    loss, max_margin_loss = 0., 0.

    for b in tqdm(range(batches_per_epoch)):
        sen_input = next(sen_gen)
        neg_input = next(neg_gen)

        batch_loss, batch_max_margin_loss = model.train_on_batch([sen_input, neg_input],
                                                                 np.ones((args.batch_size, 1))) 
        loss += batch_loss / batches_per_epoch # loss = loss + batch_loss/batches_per_epoch. 
        max_margin_loss += batch_max_margin_loss / batches_per_epoch

    tr_time = time() - t0 

    if loss < min_loss: 
        min_loss = loss
        word_emb = K.get_value(model.get_layer('word_emb').embeddings)
        aspect_emb = K.get_value(model.get_layer('aspect_emb').W)
        word_emb = word_emb / np.linalg.norm(word_emb, axis=-1, keepdims=True)
        aspect_emb = aspect_emb / np.linalg.norm(aspect_emb, axis=-1, keepdims=True)
        aspect_file = open(out_dir + '/aspect.log', 'wt', encoding='utf-8')
        model.save(out_dir + '/model_param') 
        
         
        num = 100
        res = pd.DataFrame(np.zeros(2*num).reshape(num,2))
        
        for ind in range(len(aspect_emb)):
#             desc = aspect_emb[ind]
#             sims = word_emb.dot(desc.T)
#             ordered_words = np.argsort(sims)[::-1]
#             desc_list = [vocab_inv[w] + "|" + str(sims[w]) for w in ordered_words[:100]]
#             print('Aspect %d:' % ind)
#             print(desc_list)
#             aspect_file.write('Aspect %d:\n' % ind)
#             aspect_file.write(' '.join(desc_list) + '\n\n')
        
            desc = aspect_emb[ind]
            sims = word_emb.dot(desc.T)
            ordered_words = np.argsort(sims)[::-1]
            desc_list = [vocab_inv[w] + "|" + str(sims[w]) for w in ordered_words[:num]]
            wd = [vocab_inv[w] for w in ordered_words[:num]]
            vt = [str(sims[w]) for w in ordered_words[:num]]
            asp_1 = pd.DataFrame([wd,vt]).T.rename(columns={0:'aspect_'+str(ind), 1:('aspect_'+str(ind)+'_vec')})
            res = pd.concat([res, asp_1], join = 'inner',axis=1)
            print('Aspect %d:' % ind)
            print(desc_list)
            aspect_file.write('Aspect %d:\n' % ind)
            aspect_file.write(' '.join(desc_list) + '\n\n')

    logger.info('Epoch %d, train: %is' % (ii, tr_time))
    logger.info('Total loss: %.4f, max_margin_loss: %.4f, ortho_reg: %.4f' % (loss, max_margin_loss, loss - max_margin_loss))
    
    res.to_csv('aspect_V'+str(version)+'.csv')

2019-12-05 21:17:25,047 INFO   Building model
2019-12-05 21:17:25,048 INFO Loading embeddings from: /content/drive/My Drive/Attention-Based-Aspect-Extraction-master/preprocessed_data/restaurant/w2v_embedding
2019-12-05 21:17:25,049 INFO collecting all words and their counts
2019-12-05 21:17:25,052 INFO PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-12-05 21:17:25,074 INFO collected 2956 word types from a corpus of 23707 raw words and 2603 sentences
2019-12-05 21:17:25,074 INFO Loading a fresh vocabulary
2019-12-05 21:17:25,078 INFO effective_min_count=5 retains 751 unique words (25% of original 2956, drops 2205)
2019-12-05 21:17:25,078 INFO effective_min_count=5 leaves 20174 word corpus (85% of original 23707, drops 3533)
2019-12-05 21:17:25,083 INFO deleting the raw counts dictionary of 2956 items
2019-12-05 21:17:25,084 INFO sample=0.001 downsamples 66 most-common words
2019-12-05 21:17:25,085 INFO downsampling leaves estimated 14761 word corpus (73.2% of prio

Aspect 0:
['public|0.6595423', 'local|0.6591587', 'take|0.65543026', 'conduct|0.65355146', 'controlling|0.653331', 'relate|0.65318644', 'delivery|0.65206873', 'think|0.65127414', 'especially|0.6509525', 'personal|0.6503241', 'explain|0.64772314', 'socioeconomic|0.6476651', 'co|0.6476008', 'complete|0.6470668', 'fax|0.64684224', 'order|0.6467292', 'establish|0.64663756', 'typing|0.64663553', 'sign|0.6465471', 'ethnic|0.6461451', 'group|0.646128', 'following|0.645672', 'plus|0.64566934', 'list|0.6453912', 'organized|0.64538395', 'minute|0.6451434', 'handle|0.64509946', 'resolve|0.6449853', 'request|0.6445994', 'machine|0.6445328', 'maintains|0.64434767', 'simultaneously|0.6442688', 'guest|0.64420736', 'integrity|0.644051', 'multiple|0.6439784', 'clearly|0.6438725', 'manager|0.6438634', 'type|0.6437437', 'confidentiality|0.64353085', 'exhibit|0.6432644', 'required|0.6432363', 'sale|0.64306504', 'independently|0.64288026', 'receive|0.6428786', 'initiative|0.64253306', 'location|0.6424154',

100%|██████████| 81/81 [00:01<00:00, 64.64it/s]
2019-12-05 21:17:30,802 INFO Epoch 1, train: 1s
2019-12-05 21:17:30,803 INFO Total loss: 18.1654, max_margin_loss: 18.1193, ortho_reg: 0.0460
  9%|▊         | 7/81 [00:00<00:01, 68.20it/s]

Aspect 0:
['local|0.4640023', 'ethnic|0.457528', 'socioeconomic|0.45709184', 'type|0.44395983', 'relate|0.4423694', 'public|0.44234118', 'confidentiality|0.44198444', 'complete|0.44081062', 'experienced|0.4380577', 'think|0.43701315', 'accurate|0.43683374', 'administrative|0.43654835', 'co|0.43642724', 'explain|0.43416697', 'small|0.43250978', 'especially|0.43157062', 'provider|0.43081903', 'code|0.43073946', 'machine|0.43048513', 'minute|0.4297763', 'fax|0.4291022', 'cultural|0.42890203', 'data|0.42886698', 'clearly|0.42834222', 'take|0.42695236', 'rule|0.4268596', 'receive|0.42629635', 'organizing|0.42618465', 'personal|0.425773', 'diversity|0.42426968', 'group|0.42333114', 'outlined|0.42327344', 'sign|0.42325664', 'plus|0.42302454', 'list|0.42296797', 'managing|0.42281568', 'hour|0.42248634', 'exhibit|0.42117882', 'establish|0.42112273', 'controlling|0.42072836', 'excellent|0.4197107', 'benefit|0.4196653', 'register|0.41964704', 'privacy|0.4194644', 'entry|0.41919363', 'billing|0.41

100%|██████████| 81/81 [00:01<00:00, 64.50it/s]
2019-12-05 21:17:32,212 INFO Epoch 2, train: 1s
2019-12-05 21:17:32,216 INFO Total loss: 10.1597, max_margin_loss: 10.0863, ortho_reg: 0.0735
  9%|▊         | 7/81 [00:00<00:01, 66.78it/s]

...

Aspect 0:
['plus|0.64826906', 'data|0.57338655', 'answering|0.54838026', 'proficiency|0.5464543', 'hour|0.5350448', 'benefit|0.5346379', 'specific|0.53297484', 'co|0.51794803', 'experienced|0.517456', 'establish|0.49400133', 'treatment|0.49164593', 'desire|0.48891577', 'client|0.48786986', 'diversity|0.48513836', 'carry|0.48317578', 'sufficient|0.4810755', 'xx|0.47927755', 'minute|0.476631', 'program|0.47286606', 'seattle|0.4671345', 'team|0.46671852', 'new|0.46605083', 'resolve|0.46556318', 'suite|0.46474123', 'sensitivity|0.46424603', 'ensure|0.46083218', 'month|0.45969918', 'list|0.4595724', 'utilize|0.4586377', 'personnel|0.45834902', 'conflict|0.45712432', 'animal|0.45461014', 'judgment|0.45342374', 'manual|0.45246634', 'respond|0.4521711', 'medical|0.45195508', 'day|0.45192772', 'run|0.45184886', 'make|0.45172527', 'care|0.4513058', 'quality|0.4508068', 'employer|0.44941846', 'hospital|0.4494151', 'setting|0.44936153', 'least|0.4489129', 'location|0.4462069', 'passion|0.44551402'

100%|██████████| 81/81 [00:01<00:00, 62.88it/s]
2019-12-05 21:17:47,770 INFO Epoch 13, train: 1s
2019-12-05 21:17:47,773 INFO Total loss: 0.0479, max_margin_loss: 0.0478, ortho_reg: 0.0002
  0%|          | 0/81 [00:00<?, ?it/s]

Aspect 0:
['plus|0.6416216', 'data|0.5677563', 'proficiency|0.5476427', 'answering|0.54411894', 'hour|0.5335571', 'specific|0.53099155', 'benefit|0.53018117', 'experienced|0.5181762', 'co|0.515882', 'treatment|0.49059516', 'establish|0.4901654', 'desire|0.48748282', 'diversity|0.48588833', 'carry|0.48518425', 'client|0.48447287', 'sufficient|0.48088342', 'minute|0.4754923', 'respond|0.4746708', 'team|0.4740069', 'program|0.47269467', 'new|0.4699101', 'clear|0.46798146', 'sensitivity|0.46644405', 'seattle|0.46455103', 'resolve|0.46139008', 'suite|0.4610817', 'ensure|0.45925522', 'month|0.45924777', 'list|0.4590554', 'conflict|0.45853975', 'spanish|0.45696688', 'utilize|0.4566651', 'personnel|0.45649663', 'day|0.4548142', 'xx|0.454756', 'run|0.4538999', 'animal|0.45347703', 'judgment|0.4533252', 'manual|0.45239544', 'make|0.45186502', 'care|0.4514897', 'quality|0.45126772', 'least|0.45072293', 'setting|0.44936797', 'hospital|0.44879985', 'employer|0.44864297', 'location|0.44721478', 'gre

100%|██████████| 81/81 [00:01<00:00, 64.52it/s]
2019-12-05 21:17:49,180 INFO Epoch 14, train: 1s
2019-12-05 21:17:49,192 INFO Total loss: 0.0395, max_margin_loss: 0.0394, ortho_reg: 0.0001


Aspect 0:
['plus|0.6431472', 'data|0.5641576', 'proficiency|0.5495446', 'answering|0.5462941', 'hour|0.5348194', 'specific|0.5328553', 'benefit|0.5303197', 'experienced|0.51862425', 'co|0.5173286', 'treatment|0.49276558', 'establish|0.49189752', 'clear|0.49175516', 'carry|0.4912687', 'desire|0.4890279', 'diversity|0.48751688', 'respond|0.48276216', 'sufficient|0.4823251', 'team|0.47965056', 'client|0.47880623', 'minute|0.47800517', 'new|0.47720337', 'program|0.47697788', 'seattle|0.4635194', 'spanish|0.46159992', 'suite|0.46100622', 'month|0.46066946', 'list|0.46001804', 'ensure|0.4596886', 'resolve|0.45926496', 'utilize|0.45892453', 'personnel|0.458151', 'conflict|0.4575386', 'sensitivity|0.45586085', 'day|0.45505992', 'animal|0.45456964', 'manual|0.45430428', 'judgment|0.45292953', 'run|0.45243546', 'care|0.45239374', 'least|0.45172906', 'setting|0.45167828', 'make|0.45165998', 'employer|0.45120806', 'quality|0.45082867', 'hospital|0.45081502', 'greeting|0.45001012', 'location|0.4492